In [1]:
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import keras
from tensorflow.python.keras import regularizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv1D, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from numpy import asarray
from tensorflow.keras.applications.nasnet import preprocess_input
import random
from keras import layers
from tensorflow.keras.utils import plot_model
from sklearn import model_selection
from tensorflow.keras.initializers import glorot_uniform

In [2]:
tf.config.list_physical_devices(
    device_type=None
)
path = "machine-learning-in-science-2022/"

In [3]:
def read_dataset(img_path,label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    #img = tf.image.resize(img,(224, 224))
    img = tf.cast(img, tf.float32)
    return img,label

In [4]:
def add_noise(img):
    noise = tf.random.normal(tf.shape(img),0,5)
    img += noise
    img = tf.clip_by_value(img, 0, 255)
    return img

In [5]:
df = pd.read_csv(path + 'training_norm.csv')
img_path = []
speed = []
for _,row in df.iterrows():
    img_path.append(path + 'training_data/training_data/' + str(int(row["image_id"])) + '.png')
    speed.append(int(row["speed"]))
speed = np.array(speed)

In [6]:
img_path_train,img_path_val,speed_train,speed_val = model_selection.train_test_split(img_path,speed, test_size=0.2, train_size=0.8, random_state=None, shuffle=True, stratify=None)

In [7]:
speed_one_hot_train = np.zeros((speed_train.size, speed_train.max()+1))
speed_one_hot_train[np.arange(speed_train.size),speed_train] = 1

speed_one_hot_val = np.zeros((speed_val.size, speed_val.max()+1))
speed_one_hot_val[np.arange(speed_val.size),speed_val] = 1

In [8]:
ds_train_SP = tf.data.Dataset.from_tensor_slices((img_path_train, speed_one_hot_train))
ds_train_SP = ds_train_SP.map(read_dataset).map(lambda image, label: (tf.image.random_contrast(image, lower = 0.9, upper = 1), label)
                                                                ).map(lambda image, label: (tf.image.random_saturation(image, lower = 0.8, upper = 1.2),label)
                                                                     ).map(lambda image, label: (tf.image.random_brightness(image, 0.5 ,1.5),label)
                                                                          ).map(lambda image, label: (add_noise(image), label)
                                                                               ).batch(128).repeat(4)
                                                                                   

ds_val_SP = tf.data.Dataset.from_tensor_slices((img_path_val, speed_one_hot_val))
ds_val_SP = ds_val_SP.map(read_dataset).batch(128)
                      

Metal device set to: Apple M1 Max


2022-05-02 17:12:33.067956: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-02 17:12:33.068078: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
mobile = tf.keras.applications.MobileNetV3Large(
    input_shape=(240,320,3),
    include_top=True,
    weights = None,
    classes=2,
    dropout_rate=0.2,
    classifier_activation='softmax',
    include_preprocessing=True
)
mobile.compile(optimizer="adam", loss='categorical_crossentropy',metrics = 'accuracy')
mobile.summary()

Model: "MobilenetV3large"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 240, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 240, 320, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 120, 160, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                   

In [12]:
mobile.fit(ds_train_SP,validation_data=ds_val_SP,epochs = 10)

Epoch 1/10
348/348 [==============================] - 220s 629ms/step - loss: 0.0081 - accuracy: 0.9976 - val_loss: 14.4177 - val_accuracy: 0.3842
Epoch 2/10
348/348 [==============================] - 224s 641ms/step - loss: 0.0046 - accuracy: 0.9987 - val_loss: 1.4968 - val_accuracy: 0.8978
Epoch 3/10
348/348 [==============================] - 220s 629ms/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 7.5289 - val_accuracy: 0.6245
Epoch 4/10
348/348 [==============================] - 220s 630ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.8017 - val_accuracy: 0.9217
Epoch 5/10
348/348 [==============================] - 221s 631ms/step - loss: 0.0044 - accuracy: 0.9987 - val_loss: 7.7618 - val_accuracy: 0.4995
Epoch 6/10
348/348 [==============================] - 217s 621ms/step - loss: 0.0036 - accuracy: 0.9988 - val_loss: 0.9737 - val_accuracy: 0.9297
Epoch 7/10
348/348 [==============================] - 220s 630ms/step - loss: 3.9180e-05 - accuracy: 1.0000 - val_loss: 0.3

In [ ]:
mobile = keras.models.load_model("mobile-speed")

In [ ]:
mobile.save("mobile-speed")

In [15]:
def read_dataset_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.reshape(img,[1,240, 320,3])
    return img

In [16]:
predictions = []
for i in range(1,1021):
    predictions.append(mobile.predict(read_dataset_test(path + 'test_data/test_data/' + str(i) + '.png')))

2022-05-02 19:51:28.277703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [17]:
predictions

[array([[1., 0.]], dtype=float32),
 array([[1.6475895e-14, 1.0000000e+00]], dtype=float32),
 array([[0.00986537, 0.99013466]], dtype=float32),
 array([[6.4208704e-17, 1.0000000e+00]], dtype=float32),
 array([[5.7259677e-09, 1.0000000e+00]], dtype=float32),
 array([[5.819498e-18, 1.000000e+00]], dtype=float32),
 array([[1.1932728e-17, 1.0000000e+00]], dtype=float32),
 array([[9.571623e-18, 1.000000e+00]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1.0000000e+00, 3.1133612e-21]], dtype=float32),
 array([[1.0000000e+00, 1.0040629e-26]], dtype=float32),
 array([[1.0000000e+00, 1.9011366e-22]], dtype=float32),
 array([[1.4622726e-07, 9.9999988e-01]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[5.0359033e-04, 9.9949634e-01]], dtype=float32),
 array([[0.9709646 , 0.02903541]], dtype=float32),
 array([[1.0000000e+00, 2.2263972e-32]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1.0000000e+00, 1.4302395e-34]], dtype=float32),
 array([[7.759347e-13

In [20]:
predictions[0][0][0]

1.0

In [22]:
speed_predictions = []
for i in range(len(predictions)):
    if (predictions[i][0][0] - predictions[i][0][1]) > 0.75:
        speed_predictions.append(0)
    elif (predictions[i][0][1] - predictions[i][0][0]) > 0.75:
        speed_predictions.append(1)
    else:
        speed_predictions.append(max())

In [28]:
for i in range (len(speed_predictions)):
    if speed_predictions[i] == 0.5:
        speed_predictions[i] = 1

In [29]:
df1 = pd.DataFrame(speed_predictions,columns=['speed'])

In [31]:
df1.to_excel("output_speed.xlsx")